# Installing the libraries

In [ ]:
!pip install transformers torch pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# importing the libaries

In [3]:
import os
import warnings
import logging
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

# suppressing the warnings

In [ ]:
# Suppress tokenizer warnings and W&B
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings("ignore", category=UserWarning)

# Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Check GPU
logger.info(f"CUDA available: {torch.cuda.is_available()}")

# Processing the dataset and loading mdeberta-v3-base

In [ ]:
# loading csv
data = pd.read_csv("/content/emails.csv")
data = data.dropna(subset=["email", "type"])
data = data[data["email"].str.strip() != ""]

# Validate labels
valid_labels = {"Incident", "Request", "Change", "Problem"}
label_set = set(data["type"].unique())
if not label_set.issubset(valid_labels):
    raise ValueError(f"Unexpected labels: {label_set - valid_labels}")

# Label encoding
label_map = {"Incident": 0, "Request": 1, "Change": 2, "Problem": 3}
data["label_id"] = data["type"].map(label_map)

# Split into train/test
emails = data["email"].tolist()
labels = data["label_id"].tolist()
train_emails, test_emails, train_labels, test_labels = train_test_split(
    emails, labels, test_size=0.1, stratify=labels, random_state=42
)

# Load tokenizer & model
model_name = "microsoft/mdeberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Tokenization
logger.info("Tokenizing...")
train_encodings = tokenizer(train_emails, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_emails, truncation=True, padding=True, max_length=512)


# Dataset class
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = EmailDataset(train_encodings, train_labels)
test_dataset = EmailDataset(test_encodings, test_labels)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# computing class weights

In [ ]:

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

train_labels = data["label_id"].values  

classes = np.unique(train_labels)  # [0,1,2,3]
weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=train_labels
)
class_weights = torch.tensor(weights, dtype=torch.float)

print("Per-class weights:", class_weights)

Per-class weights: tensor([0.6259, 0.8746, 2.3838, 1.1912])


# Trainer with weighted classes

In [ ]:
from torch.nn import CrossEntropyLoss

class WeightedTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.model.device)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Training Process

In [ ]:
training_args = TrainingArguments(
    output_dir="/content",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="/content/logs",
    logging_steps=100,
    save_steps=3600,
    fp16=True,
    report_to="none",
    dataloader_num_workers=2,
)

# trainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    class_weights=class_weights,
)

# Training
logger.info("Training started...")
trainer.train()
logger.info("Training completed.")

Step,Training Loss
100,1.370100
200,1.098400
300,0.725600
400,0.655000
500,0.657600
600,0.732900
700,0.756800
800,0.641300
900,0.624400
1000,0.655200


# saving model in zip for downloading

In [ ]:
import os

model_dir = "/content/mdeberta_finetuned_model"
tokenizer_dir = "/content/mdeberta_finetuned_tokenizer"

model.save_pretrained(model_dir)
tokenizer.save_pretrained(tokenizer_dir)

os.system(f"zip -r {model_dir}.zip {model_dir}")
os.system(f"zip -r {tokenizer_dir}.zip {tokenizer_dir}")
logger.info("Model and tokenizer saved and zipped.")

Zipped model to: /content/mdeberta_finetuned_model.zip
Zipped tokenizer to: /content/mdeberta_finetuned_tokenizer.zip


# comaparing zeroshot vs finetuned

In [ ]:
import numpy as np
import logging
from transformers import Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report

# 1. Label
label_map = {"Incident": 0, "Request": 1, "Change": 2, "Problem": 3}
num_labels = len(label_map)  # ← define num_labels here

# 2. Encode labels in data
data["label_id"] = data["type"].map(label_map)

# Paths and model names
model_name = "microsoft/mdeberta-v3-base"
model_dir = "/content/mdeberta_finetuned_model"

# 1. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2a. Zero‑shot model
zero_shot_model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels
)
zero_shot_model.eval()

# 2b. Fine‑tuned model
finetuned_model = AutoModelForSequenceClassification.from_pretrained(model_dir)
finetuned_model.eval()

# 3. Trainers
zero_shot_trainer = Trainer(model=zero_shot_model, tokenizer=tokenizer)
finetuned_trainer = Trainer(model=finetuned_model, tokenizer=tokenizer)


# 4. Prediction helper
def get_predictions(trainer, dataset):
    raw_preds, _, _ = trainer.predict(dataset)
    return np.argmax(raw_preds, axis=1)


# 5. inference on test set
zero_preds = get_predictions(zero_shot_trainer, test_dataset)
fine_preds = get_predictions(finetuned_trainer, test_dataset)

# 6. Evaluation
print("=== Zero‑Shot mDeBERTa‑v3 ===")
print(f"Accuracy: {accuracy_score(test_labels, zero_preds):.4f}")
print(
    classification_report(test_labels, zero_preds, target_names=list(label_map.keys()))
)

print("\n=== Fine‑Tuned mDeBERTa‑v3 ===")
print(f"Accuracy: {accuracy_score(test_labels, fine_preds):.4f}")
print(
    classification_report(test_labels, fine_preds, target_names=list(label_map.keys()))
)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/mdeberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-e4ec807c3d9a>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  zero_shot_trainer = Trainer(model=zero_shot_model, tokenizer=tokenizer)
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-18-e4ec807c3d9a>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  finetuned_trainer = Trainer(model=

=== Zero‑Shot mDeBERTa‑v3 ===
Accuracy: 0.3992
              precision    recall  f1-score   support

    Incident       0.40      1.00      0.57       958
     Request       0.00      0.00      0.00       686
      Change       0.00      0.00      0.00       252
     Problem       0.00      0.00      0.00       504

    accuracy                           0.40      2400
   macro avg       0.10      0.25      0.14      2400
weighted avg       0.16      0.40      0.23      2400


=== Fine‑Tuned mDeBERTa‑v3 ===
Accuracy: 0.7971
              precision    recall  f1-score   support

    Incident       0.75      0.78      0.77       958
     Request       0.95      0.96      0.95       686
      Change       0.97      0.91      0.94       252
     Problem       0.58      0.55      0.57       504

    accuracy                           0.80      2400
   macro avg       0.81      0.80      0.81      2400
weighted avg       0.80      0.80      0.80      2400

